In [55]:
# 1-) Yeni Urunlerin Sektorleri.xlsx

import pandas as pd

def load_and_process_new_products():
    # Kullanacağımız yılları tanımlıyoruz
    years = [2002, 2007, 2012, 2017, 2022]
    file_paths = {
        2002: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS02_V202401\product_codes_HS02_V202401.csv",
        2007: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv",
        2012: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv",
        2017: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv",
        2022: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv"
    }
    
    # Her yıl için ürün kodlarını ekliyoruz
    dfs = {year: pd.read_csv(file_paths[year]) for year in years}
    product_sets = {year: set(dfs[year]['code']) for year in years}

    # Yeni ürünleri buluyoruz
    new_products_dfs = []
    for i in range(1, len(years)):
        year = years[i]
        prev_year = years[i - 1]
        new_products = product_sets[year] - product_sets[prev_year]
        
        # Yıl kolonunu ekliyoruz
        new_df = dfs[year][dfs[year]['code'].isin(new_products)].copy()
        new_df['Yıl'] = year
        new_products_dfs.append(new_df)
    
    # Bütün yıllardaki yeni ürünleri tek dataframede topluyoruz
    combined_new_products = pd.concat(new_products_dfs, ignore_index=True)
    
    # 2 basamaklı HS kodunu hesaplıyoruz
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000
    
    # Sektör kodlarını ve isimilerini kullanarak birleştiriyoruz
    atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
    atlas_df['hs2'] = atlas_df['hs2'].astype(str).str.zfill(2)
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2)
    combined_new_products = combined_new_products.merge(
        atlas_df[['hs2', 'hs2_name', 'sector']],
        left_on='2 Basamaklı HS Kodu',
        right_on='hs2',
        how='left'
    ).rename(columns={
        'code': '6 Basamaklı HS Kodu',
        'hs2_name': '2 Basamaklı HS İsmi',
        'sector': 'Sektör İsmi'
    }).drop(columns=['hs2'])

    # Yıl sütununu en başa alıyoruz
    cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
    combined_new_products = combined_new_products[cols]
    return combined_new_products

# Fonksiyonu uyguluyoruz
processed_new_products = load_and_process_new_products()

# Sadece Sektör İsmi 'Textiles' olanları filtreliyoruz
textiles_only = processed_new_products[processed_new_products['Sektör İsmi'] == 'Textiles ']

# "4 Basamaklı HS Kodu" sütunu ekliyoruz
textiles_only['4 Basamaklı HS Kodu'] = textiles_only['6 Basamaklı HS Kodu'].astype(str).str[:4]

# Kaydediyoruz
textiles_only.to_excel("1-) Yeni Urunlerin Sektorleri.xlsx", index=False)

C:\Users\oguzo\AppData\Local\Temp\ipykernel_13368\158877336.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textiles_only['4 Basamaklı HS Kodu'] = textiles_only['6 Basamaklı HS Kodu'].astype(str).str[:4]


In [3]:
# Toplu datayı birleştirme tekrar çağırarak vakit kaybetmemek için

# Yıllar, HS versiyonları ve dosya yolu formatı
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}
dosya_yolu_format = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Atlas dosyasını oku ve sütunları temizle
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
combined_atlas_df['hs2_name'] = combined_atlas_df['hs2_name'].str.strip()

# Boş bir liste başlangıçta tutuyoruz
all_data = []

# Her yılın verisini işle ve listeye ekle
for year in yillar:
    hs_version = hs_versiyonlari[year]
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
    print(f"İşleniyor: {dosya_yolu}")
    df = pd.read_csv(dosya_yolu)
    
    # HS2 ve HS4 kodunu hesapla ve sektörel bilgilerle birleştir
    df['hs2'] = df['k'] // 10000  
    df = df.merge(combined_atlas_df, on='hs2', how='left')  
    
    # Yıla göre birleştirilen veriyi işaretleyelim
    df['year'] = year
    
    # Listeye ekle
    all_data.append(df)

# Tüm yılları birleştir
merged_df = pd.concat(all_data, ignore_index=True)

# Elde edilen veri çerçevesini kontrol et
merged_textiles_df = merged_df[merged_df["sector"] == "Textiles"]
merged_textiles_df["hs4"] = merged_textiles_df["k"] // 100
merged_df["hs4"] = merged_df["k"] // 100

İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2012_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2013_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2014_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2015_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2016_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2017_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2018_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2019_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2020_V202

C:\Users\oguzo\AppData\Local\Temp\ipykernel_13368\1206094459.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_textiles_df["hs4"] = merged_textiles_df["k"] // 100


In [7]:
# HS2 textiles listesini belirliyoruz

# Çağırıp boşlukları temizliyoruz
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2', 'hs2_name']]

# Kodları bir listeye koyuyoruz
textiles_hs2_codes = textiles_hs2['hs2'].dropna().unique().tolist()

# İsimleriyle beraber bir sözlük oluşturuyoruz
textiles_hs2_dict = textiles_hs2.set_index('hs2')['hs2_name'].str.strip().to_dict()

In [16]:
# 2-) Turkiye'de tekstil ticaret hacmi 

# Türkiye kodu
turkey_code = 792

# Türkiye'nin dahil olduğu tüm ticaret
turkiye_ticaret = merged_df[
    (merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code)
]

# Türkiye'nin tekstil ticareti
turkiye_textiles = merged_df[
    (merged_df['sector'] == 'Textiles') & 
    ((merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code))
]

# Yıllara göre toplam ticaret hacmini hesaplıyoruz
toplam_ticaret_hacmi = turkiye_ticaret.groupby('year')['v'].sum().reset_index()
textiles_ticaret_hacmi = turkiye_textiles.groupby('year')['v'].sum().reset_index()

# Tekstil ticaretinin toplam ticarete oranını hesaplıyoruz
oran_df = toplam_ticaret_hacmi.copy()
oran_df['Türkiyenin Toplam Tekstil Ticaret Hacmi'] = textiles_ticaret_hacmi['v']
oran_df['Tekstil Oranı (%)'] = (oran_df['Türkiyenin Toplam Tekstil Ticaret Hacmi'] / oran_df['v']) * 100
oran_df.rename(columns={"v": "Türkiyenin Toplam Ticaret Hacmi"}, inplace=True)
oran_df.rename(columns={"year": "Yıl"}, inplace=True)
pd.options.display.float_format = '{:,.2f}'.format

# Excel dosyasına kaydediyoruz
oran_df.to_excel("2-) Turkiye'de tekstil ticaret hacmi.xlsx", index=False)

In [24]:
# 3-) Turkiye'de tekstil ithalat ve ihracat hacmi 

import pandas as pd

# Türkiye kodu
turkey_code = 792

# Türkiye'nin dahil olduğu tüm ticaret
turkiye_ticaret = merged_df[
    (merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code)
]

# Türkiye'nin toplam ihracat hacmi 
toplam_ihracat_hacmi = turkiye_ticaret[turkiye_ticaret['i'] == turkey_code].groupby('year')['v'].sum().reset_index()
toplam_ihracat_hacmi.rename(columns={'v': 'Türkiyenin Toplam İhracat Hacmi'}, inplace=True)

# Türkiye'nin toplam ithalat hacmi 
toplam_ithalat_hacmi = turkiye_ticaret[turkiye_ticaret['j'] == turkey_code].groupby('year')['v'].sum().reset_index()
toplam_ithalat_hacmi.rename(columns={'v': 'Türkiyenin Toplam İthalat Hacmi'}, inplace=True)

# Türkiye'nin tekstil ihracatı 
turkiye_textile_export = merged_df[
    (merged_df['sector'] == 'Textiles') & (merged_df['i'] == turkey_code)
]
textile_export_hacmi = turkiye_textile_export.groupby('year')['v'].sum().reset_index()
textile_export_hacmi.rename(columns={'v': 'Tekstil İhracat Hacmi'}, inplace=True)

# Türkiye'nin tekstil ithalatı 
turkiye_textile_import = merged_df[
    (merged_df['sector'] == 'Textiles') & (merged_df['j'] == turkey_code)
]
textile_import_hacmi = turkiye_textile_import.groupby('year')['v'].sum().reset_index()
textile_import_hacmi.rename(columns={'v': 'Tekstil İthalat Hacmi'}, inplace=True)

# Tüm verileri birleştiriyoruz
oran_df = toplam_ihracat_hacmi.merge(textile_export_hacmi, on='year', how='left')
oran_df = oran_df.merge(toplam_ithalat_hacmi, on='year', how='left')
oran_df = oran_df.merge(textile_import_hacmi, on='year', how='left')

# Boş değerleri sıfırla dolduruyoruz
oran_df['Tekstil İhracat Hacmi'] = oran_df['Tekstil İhracat Hacmi'].fillna(0)
oran_df['Tekstil İthalat Hacmi'] = oran_df['Tekstil İthalat Hacmi'].fillna(0)

# Oranları hesaplıyoruz
oran_df['Tekstil İhracat Oranı (%)'] = (oran_df['Tekstil İhracat Hacmi'] / oran_df['Türkiyenin Toplam İhracat Hacmi']) * 100
oran_df['Tekstil İthalat Oranı (%)'] = (oran_df['Tekstil İthalat Hacmi'] / oran_df['Türkiyenin Toplam İthalat Hacmi']) * 100

# Sütun isimlerini düzenle
oran_df.rename(columns={'year': 'Yıl'}, inplace=True)
pd.options.display.float_format = '{:,.2f}'.format

# Excel dosyasına kaydediyoruz
oran_df.to_excel("3-) Turkiye'de tekstil ithalat ve ihracat hacmi.xlsx", index=False)

In [30]:
# 4-) Dunyada tekstil ticaret hacmi

# Küresel toplam ticaret hacmi
global_trade = merged_df.groupby('year')['v'].sum().reset_index()
global_trade.rename(columns={'v': 'Küresel Toplam Ticaret Hacmi'}, inplace=True)

# Küresel tekstil ticaret hacmi 
global_textile_trade = merged_df[merged_df['sector'] == 'Textiles']
global_textile = global_textile_trade.groupby('year')['v'].sum().reset_index()
global_textile.rename(columns={'v': 'Küresel Tekstil Ticaret Hacmi'}, inplace=True)

# Verileri birleştir ve boşları sıfırla
global_trade_summary = global_trade.merge(global_textile, on='year', how='left')
global_trade_summary['Küresel Tekstil Ticaret Hacmi'] = global_trade_summary['Küresel Tekstil Ticaret Hacmi'].fillna(0)

# Oranını hesaplıyoruz
global_trade_summary['Tekstil Ticaret Oranı (%)'] = (global_trade_summary['Küresel Tekstil Ticaret Hacmi'] / 
                                                    global_trade_summary['Küresel Toplam Ticaret Hacmi']) * 100

# Yıl sütununu düzenliyoruz
global_trade_summary.rename(columns={'year': 'Yıl'}, inplace=True)
pd.options.display.float_format = '{:,.2f}'.format

# Veriyi Excel'e kaydet
global_trade_summary.to_excel("4-) Dunyada tekstil ticaret hacmi.xlsx", index=False)

In [38]:
# 5-) Turkiye'nin tekstilde ihracat ve ithalat partnerleri


# Ülke kodları dosyasını ekliyoruz
turkey_code = 792
country_codes = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\country_codes_V202401.csv"
countries = pd.read_csv(country_codes)

# Tekstil sektörünü filtreliyoruz
textile_trade = merged_df[merged_df['sector'] == 'Textiles']

# Excel yazıcı oluştur
with pd.ExcelWriter("5-) Turkiye'nin tekstilde ihracat ve ithalat partnerleri.xlsx") as writer:
    for year in range(2012, 2022):  # 2012'den 2021'e kadar
        # Belirli yıl için filtreleme
        year_trade = textile_trade[textile_trade['year'] == year]
        
        # Türkiye'nin tekstil ihracatı (i = Türkiye)
        turkiye_textile_export = year_trade[year_trade['i'] == turkey_code]
        export_by_country = turkiye_textile_export.groupby('j')['v'].sum().reset_index()
        export_by_country.rename(columns={'j': 'country_code', 'v': 'İhracat Değeri'}, inplace=True)
        
        # Türkiye'nin tekstil ithalatı (j = Türkiye)
        turkiye_textile_import = year_trade[year_trade['j'] == turkey_code]
        import_by_country = turkiye_textile_import.groupby('i')['v'].sum().reset_index()
        import_by_country.rename(columns={'i': 'country_code', 'v': 'İthalat Değeri'}, inplace=True)
        
        # Toplam ihracat ve ithalat hacimlerini hesapla
        toplam_ihracat = export_by_country['İhracat Değeri'].sum()
        toplam_ithalat = import_by_country['İthalat Değeri'].sum()
        
        # Oranları hesapla
        export_by_country['İhracat Oranı (%)'] = (export_by_country['İhracat Değeri'] / toplam_ihracat) * 100
        import_by_country['İthalat Oranı (%)'] = (import_by_country['İthalat Değeri'] / toplam_ithalat) * 100
        
        # Ülke isimlerini ekliyoruz
        export_by_country = export_by_country.merge(countries, on='country_code', how='left')
        import_by_country = import_by_country.merge(countries, on='country_code', how='left')
        export_by_country.rename(columns={'country_name': 'Ülke Adı'}, inplace=True)
        import_by_country.rename(columns={'country_name': 'Ülke Adı'}, inplace=True)
        
        # En çok ihracat ve ithalat yapılan 40 ülkeyi seçiyoruz
        top40_export = export_by_country.sort_values(by='İhracat Değeri', ascending=False).head(40)
        top40_import = import_by_country.sort_values(by='İthalat Değeri', ascending=False).head(40)
        
        # Sonuçları Excel'e kaydediyoruz (Yıl bazında ayrı sayfalar)
        top40_export[['Ülke Adı', 'İhracat Değeri', 'İhracat Oranı (%)']].to_excel(writer, sheet_name=f'{year}_İhracat', index=False)
        top40_import[['Ülke Adı', 'İthalat Değeri', 'İthalat Oranı (%)']].to_excel(writer, sheet_name=f'{year}_İthalat', index=False)

In [40]:
# 6-) Dunyada tekstilde en buyuk ihracatcilar ve ithalatcilar

# Ülke kodları dosyasını ekliyoruz
country_codes = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\country_codes_V202401.csv"
countries = pd.read_csv(country_codes)

# Tekstil sektörünü filtreliyoruz
textile_trade = merged_df[merged_df['sector'] == 'Textiles']

# Excel yazıcı oluştur
with pd.ExcelWriter("6-) Dunyada tekstilde en buyuk ihracatcilar ve ithalatcilar.xlsx") as writer:
    for year in range(2012, 2022):  # 2012'den 2021'e kadar
        # Belirli yıl için filtreleme
        year_trade = textile_trade[textile_trade['year'] == year]

        # Global tekstil ihracatı 
        global_textile_export = year_trade.groupby('i')['v'].sum().reset_index()
        global_textile_export.rename(columns={'i': 'country_code', 'v': 'İhracat Değeri'}, inplace=True)

        # Global tekstil ithalatı 
        global_textile_import = year_trade.groupby('j')['v'].sum().reset_index()
        global_textile_import.rename(columns={'j': 'country_code', 'v': 'İthalat Değeri'}, inplace=True)

        # Toplam ihracat ve ithalat hacimlerini hesaplıyoruz
        toplam_global_ihracat = global_textile_export['İhracat Değeri'].sum()
        toplam_global_ithalat = global_textile_import['İthalat Değeri'].sum()

        # Oranları hesaplıyoruz
        global_textile_export['İhracat Oranı (%)'] = (global_textile_export['İhracat Değeri'] / toplam_global_ihracat) * 100
        global_textile_import['İthalat Oranı (%)'] = (global_textile_import['İthalat Değeri'] / toplam_global_ithalat) * 100

        # Ülke isimlerini ekliyoruz
        global_textile_export = global_textile_export.merge(countries, on='country_code', how='left')
        global_textile_import = global_textile_import.merge(countries, on='country_code', how='left')
        global_textile_export.rename(columns={'country_name': 'Ülke Adı'}, inplace=True)
        global_textile_import.rename(columns={'country_name': 'Ülke Adı'}, inplace=True)

        # En çok ihracat ve ithalat yapan 40 ülkeyi seçiyoruz
        top40_export = global_textile_export.sort_values(by='İhracat Değeri', ascending=False).head(40)
        top40_import = global_textile_import.sort_values(by='İthalat Değeri', ascending=False).head(40)

        # Excel'e kaydediyoruz
        top40_export[['Ülke Adı', 'İhracat Değeri', 'İhracat Oranı (%)']].to_excel(writer, sheet_name=f'{year}_İhracat', index=False)
        top40_import[['Ülke Adı', 'İthalat Değeri', 'İthalat Oranı (%)']].to_excel(writer, sheet_name=f'{year}_İthalat', index=False)

In [45]:
# 7-) Turkiye'de tekstilde HS2 ihracat ve ithalat kirilimlari

# HS2 isimlerinin Türkçe çevirilerini tanımlıyoruz (İnternette bulamadım manuel oldu)
hs2_turkce_ceviri = {
    "Silk": "İpek",
    "Wool": "Yün",
    "Cotton": "Pamuk",
    "Other vegetable textile fibres": "Diğer bitkisel tekstil lifleri",
    "Man-made filaments": "Yapay filamentler",
    "Man-made staple fibres": "Yapay kesikli lifler",
    "Wadding, felt and nonwovens": "Vatka, keçe ve dokusuz yüzeyler",
    "Carpets": "Halılar",
    "Special woven fabrics": "Özel dokuma kumaşlar",
    "Impregnated, coated or laminated textile fabrics": "Emdirilmiş, kaplanmış veya lamine kumaşlar",
    "Knitted fabrics": "Örme kumaşlar",
    "Apparel, knit": "Örme giyim",
    "Apparel, not knit": "Örme olmayan giyim",
    "Other made up textile articles": "Diğer hazır tekstil ürünleri",
    "Footwear": "Ayakkabılar",
    "Headgear": "Başlıklar",
    "Umbrellas and walking-sticks": "Şemsiyeler ve bastonlar",
    "Feathers and down": "Tüyler ve kuş tüyleri",
    "Furniture": "Mobilya",
    "Art": "Sanat"
}

# Türkiye kodu ve dosyaları yüklüyoruz
turkey_code = 792
file_path = '0-) HS92 Sektor Kodlari ve Isimleri.xlsx'
country_codes_file = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\country_codes_V202401.csv"
combined_atlas_df = pd.read_excel(file_path)
country_codes_df = pd.read_csv(country_codes_file)

# Tekstil HS2 kodlarını çekiyoruz
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2', 'hs2_name']]
hs2_list = textiles_hs2['hs2'].dropna().unique().tolist()
hs2_dict = textiles_hs2.set_index('hs2')['hs2_name'].str.strip().to_dict()

# Tekstil sektörüne ait veriyi filtreliyoruz
turkiye_textiles = merged_df[
    (merged_df['hs2'].isin(hs2_list)) & 
    ((merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code))
]

# Türkiye'nin HS2 bazında ihracatını hesaplıyoruz
ihracat_hs2 = turkiye_textiles[turkiye_textiles['i'] == turkey_code].groupby(['year', 'hs2'])['v'].sum().reset_index()
ihracat_hs2.rename(columns={'v': 'İhracat Değeri'}, inplace=True)
ihracat_hs2['HS2 İsmi'] = ihracat_hs2['hs2'].map(hs2_dict)

# Türkiye'nin HS2 bazında ithalatını hesaplıyoruz
ithalat_hs2 = turkiye_textiles[turkiye_textiles['j'] == turkey_code].groupby(['year', 'hs2'])['v'].sum().reset_index()
ithalat_hs2.rename(columns={'v': 'İthalat Değeri'}, inplace=True)
ithalat_hs2['HS2 İsmi'] = ithalat_hs2['hs2'].map(hs2_dict)

# Toplam tekstil ihracat ve ithalat hacimlerini hesaplıyoruz
toplam_textile_ihracat = ihracat_hs2.groupby('year')['İhracat Değeri'].sum().reset_index()
toplam_textile_ithalat = ithalat_hs2.groupby('year')['İthalat Değeri'].sum().reset_index()

# Oranları hesaplıyoruz (İhracat)
ihracat_hs2 = ihracat_hs2.merge(toplam_textile_ihracat, on='year', how='left', suffixes=('', '_toplam'))
ihracat_hs2['İhracat Oranı (%)'] = (ihracat_hs2['İhracat Değeri'] / ihracat_hs2['İhracat Değeri_toplam']) * 100

# Oranları hesaplıyoruz (İthalat)
ithalat_hs2 = ithalat_hs2.merge(toplam_textile_ithalat, on='year', how='left', suffixes=('', '_toplam'))
ithalat_hs2['İthalat Oranı (%)'] = (ithalat_hs2['İthalat Değeri'] / ithalat_hs2['İthalat Değeri_toplam']) * 100

# Türkçelerini ekliyoruz maple
ihracat_hs2['HS2 İsmi Türkçe'] = ihracat_hs2['HS2 İsmi'].map(hs2_turkce_ceviri)
ithalat_hs2['HS2 İsmi Türkçe'] = ithalat_hs2['HS2 İsmi'].map(hs2_turkce_ceviri)

# Yıl sütununu ayarlıyoruz
ihracat_hs2.rename(columns={'year': 'Yıl', 'hs2': 'HS2'}, inplace=True)
ithalat_hs2.rename(columns={'year': 'Yıl', 'hs2': 'HS2'}, inplace=True)

# Excel'e kaydediyoruz
with pd.ExcelWriter("7-) Turkiye'de tekstilde HS2 ihracat ve ithalat kirilimlari.xlsx") as writer:
    ihracat_hs2[['Yıl', 'HS2', 'HS2 İsmi', 'HS2 İsmi Türkçe', 'İhracat Değeri', 'İhracat Oranı (%)']].to_excel(writer, sheet_name='HS2 Bazında İhracat', index=False)
    ithalat_hs2[['Yıl', 'HS2', 'HS2 İsmi', 'HS2 İsmi Türkçe', 'İthalat Değeri', 'İthalat Oranı (%)']].to_excel(writer, sheet_name='HS2 Bazında İthalat', index=False)

In [46]:
# 8-) Turkiye'de tekstilde HS4 ihracat ve ithalat kirilimlari.xlsx

import pandas as pd

# Türkiye kodu
turkey_code = 792

combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# "sector" sütunundaki boşlukları temizle ve sadece "Textiles" sektörü için filtrele
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2']]

# HS2 kodlarını al ve HS4 seviyesinde analiz için uygun hale getir
hs2_list = textiles_hs2['hs2'].dropna().unique().tolist()

# Tekstil sektörüne ait veriyi filtrele ve HS4 seviyesine dönüştür
turkiye_textiles = merged_df[
    (merged_df['hs2'].isin(hs2_list)) &
    ((merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code))
]
turkiye_textiles['HS4'] = turkiye_textiles['k'] // 100  # HS4 kodu: İlk 4 hane

# Türkiye'nin HS4 bazında ihracatını hesapla
ihracat_hs4 = turkiye_textiles[turkiye_textiles['i'] == turkey_code].groupby(['year', 'HS4'])['v'].sum().reset_index()
ihracat_hs4.rename(columns={'v': 'İhracat Değeri'}, inplace=True)

# Türkiye'nin HS4 bazında ithalatını hesapla
ithalat_hs4 = turkiye_textiles[turkiye_textiles['j'] == turkey_code].groupby(['year', 'HS4'])['v'].sum().reset_index()
ithalat_hs4.rename(columns={'v': 'İthalat Değeri'}, inplace=True)

# Toplam tekstil ihracat ve ithalat hacimlerini hesapla
toplam_textile_ihracat = ihracat_hs4.groupby('year')['İhracat Değeri'].sum().reset_index()
toplam_textile_ithalat = ithalat_hs4.groupby('year')['İthalat Değeri'].sum().reset_index()

# Oranları hesapla (İhracat)
ihracat_hs4 = ihracat_hs4.merge(toplam_textile_ihracat, on='year', how='left', suffixes=('', '_toplam'))
ihracat_hs4['İhracat Oranı (%)'] = (ihracat_hs4['İhracat Değeri'] / ihracat_hs4['İhracat Değeri_toplam']) * 100

# Oranları hesapla (İthalat)
ithalat_hs4 = ithalat_hs4.merge(toplam_textile_ithalat, on='year', how='left', suffixes=('', '_toplam'))
ithalat_hs4['İthalat Oranı (%)'] = (ithalat_hs4['İthalat Değeri'] / ithalat_hs4['İthalat Değeri_toplam']) * 100

# Sütun isimlerini düzenle
ihracat_hs4.rename(columns={'year': 'Yıl'}, inplace=True)
ithalat_hs4.rename(columns={'year': 'Yıl'}, inplace=True)

# Sonuçları Excel'e kaydet
with pd.ExcelWriter("8-) Turkiye'de tekstilde HS4 ihracat ve ithalat kirilimlari.xlsx") as writer:
    ihracat_hs4[['Yıl', 'HS4', 'İhracat Değeri', 'İhracat Oranı (%)']].to_excel(writer, sheet_name='HS4 Bazında İhracat', index=False)
    ithalat_hs4[['Yıl', 'HS4', 'İthalat Değeri', 'İthalat Oranı (%)']].to_excel(writer, sheet_name='HS4 Bazında İthalat', index=False)

C:\Users\oguzo\AppData\Local\Temp\ipykernel_13368\1902441363.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turkiye_textiles['HS4'] = turkiye_textiles['k'] // 100  # HS4 kodu: İlk 4 hane


In [49]:
# 9-) Dunyada tekstilde HS2 ihracat ve ithalat kirilimlari

# HS2 isimlerinin türkçe çevirilerini tanımlıyoruz
hs2_turkce_ceviri = {
    "Silk": "İpek",
    "Wool": "Yün",
    "Cotton": "Pamuk",
    "Other vegetable textile fibres": "Diğer bitkisel tekstil lifleri",
    "Man-made filaments": "Yapay filamentler",
    "Man-made staple fibres": "Yapay kesikli lifler",
    "Wadding, felt and nonwovens": "Vatka, keçe ve dokusuz yüzeyler",
    "Carpets": "Halılar",
    "Special woven fabrics": "Özel dokuma kumaşlar",
    "Impregnated, coated or laminated textile fabrics": "Emdirilmiş, kaplanmış veya lamine kumaşlar",
    "Knitted fabrics": "Örme kumaşlar",
    "Apparel, knit": "Örme giyim",
    "Apparel, not knit": "Örme olmayan giyim",
    "Other made up textile articles": "Diğer hazır tekstil ürünleri",
    "Footwear": "Ayakkabılar",
    "Headgear": "Başlıklar",
    "Umbrellas and walking-sticks": "Şemsiyeler ve bastonlar",
    "Feathers and down": "Tüyler ve kuş tüyleri",
    "Furniture": "Mobilya",
    "Art": "Sanat"
}

# Dosyayı yüklüyoruz
file_path = '0-) HS92 Sektor Kodlari ve Isimleri.xlsx'
combined_atlas_df = pd.read_excel(file_path)

# Tekstil HS2 kodlarını çekiyoruz
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2', 'hs2_name']]
hs2_list = textiles_hs2['hs2'].dropna().unique().tolist()
hs2_dict = textiles_hs2.set_index('hs2')['hs2_name'].str.strip().to_dict()

# Tekstil sektörüne ait veriyi filtreliyoruz
global_textiles = merged_df[merged_df['hs2'].isin(hs2_list)]

# Global düzeyde HS2 bazında ihracatı hesaplıyoruz
global_ihracat_hs2 = global_textiles.groupby(['year', 'hs2'])['v'].sum().reset_index()
global_ihracat_hs2.rename(columns={'v': 'Global İhracat Değeri'}, inplace=True)
global_ihracat_hs2['HS2 İsmi'] = global_ihracat_hs2['hs2'].map(hs2_dict)

# Global düzeyde HS2 bazında ithalatı hesaplıyoruz
global_ithalat_hs2 = global_textiles.groupby(['year', 'hs2'])['v'].sum().reset_index()
global_ithalat_hs2.rename(columns={'v': 'Global İthalat Değeri'}, inplace=True)
global_ithalat_hs2['HS2 İsmi'] = global_ithalat_hs2['hs2'].map(hs2_dict)

# Toplam global tekstil ihracat ve ithalat hacimlerini hesaplıyoruz
toplam_global_textile_ihracat = global_ihracat_hs2.groupby('year')['Global İhracat Değeri'].sum().reset_index()
toplam_global_textile_ithalat = global_ithalat_hs2.groupby('year')['Global İthalat Değeri'].sum().reset_index()

# Oranları hesaplıyoruz (İhracat)
global_ihracat_hs2 = global_ihracat_hs2.merge(toplam_global_textile_ihracat, on='year', how='left', suffixes=('', '_toplam'))
global_ihracat_hs2['Global İhracat Oranı (%)'] = (global_ihracat_hs2['Global İhracat Değeri'] / global_ihracat_hs2['Global İhracat Değeri_toplam']) * 100

# Oranları hesaplıyoruz (İthalat)
global_ithalat_hs2 = global_ithalat_hs2.merge(toplam_global_textile_ithalat, on='year', how='left', suffixes=('', '_toplam'))
global_ithalat_hs2['Global İthalat Oranı (%)'] = (global_ithalat_hs2['Global İthalat Değeri'] / global_ithalat_hs2['Global İthalat Değeri_toplam']) * 100

# Türkçelerini ekliyoruz
global_ihracat_hs2['HS2 İsmi Türkçe'] = global_ihracat_hs2['HS2 İsmi'].map(hs2_turkce_ceviri)
global_ithalat_hs2['HS2 İsmi Türkçe'] = global_ithalat_hs2['HS2 İsmi'].map(hs2_turkce_ceviri)

# Yıl sütununu düzenliyoruz
global_ihracat_hs2.rename(columns={'year': 'Yıl', 'hs2': 'HS2'}, inplace=True)
global_ithalat_hs2.rename(columns={'year': 'Yıl', 'hs2': 'HS2'}, inplace=True)

# Excel'e kaydediyoruz
with pd.ExcelWriter("9-) Dunyada tekstilde HS2 ihracat ve ithalat kirilimlari.xlsx") as writer:
    global_ihracat_hs2[['Yıl', 'HS2', 'HS2 İsmi', 'HS2 İsmi Türkçe', 'Global İhracat Değeri', 'Global İhracat Oranı (%)']].to_excel(writer, sheet_name='HS2 Bazında İhracat', index=False)
    global_ithalat_hs2[['Yıl', 'HS2', 'HS2 İsmi', 'HS2 İsmi Türkçe', 'Global İthalat Değeri', 'Global İthalat Oranı (%)']].to_excel(writer, sheet_name='HS2 Bazında İthalat', index=False)

In [53]:
# 10-) Dunyada tekstilde HS4 ihracat ve ithalat kirilimlari

# Sektör Kodlarını yüklüyoruz
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Boşlukları temizleyip, tekstili filtreliyoruz
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2']]

# Tekstil sektörüne ait veriyi filtrele ve HS4 seviyesine ayarla
hs2_list = textiles_hs2['hs2'].dropna().unique().tolist()
global_textiles = merged_df[merged_df['hs2'].isin(hs2_list)]
global_textiles['HS4'] = global_textiles['k'] // 100 

# Global düzeyde HS4 bazında ihracatı hesaplıyoruz
global_ihracat_hs4 = global_textiles.groupby(['year', 'HS4'])['v'].sum().reset_index()
global_ihracat_hs4.rename(columns={'v': 'Global İhracat Değeri'}, inplace=True)

# Global düzeyde HS4 bazında ithalatı hesaplıyoruz
global_ithalat_hs4 = global_textiles.groupby(['year', 'HS4'])['v'].sum().reset_index()
global_ithalat_hs4.rename(columns={'v': 'Global İthalat Değeri'}, inplace=True)

# Toplam global tekstil ihracat ve ithalat hacimlerini hesaplıyoruz
toplam_global_textile_ihracat = global_ihracat_hs4.groupby('year')['Global İhracat Değeri'].sum().reset_index()
toplam_global_textile_ithalat = global_ithalat_hs4.groupby('year')['Global İthalat Değeri'].sum().reset_index()

# Oranları hesapla (İhracat)
global_ihracat_hs4 = global_ihracat_hs4.merge(toplam_global_textile_ihracat, on='year', how='left', suffixes=('', '_toplam'))
global_ihracat_hs4['Global İhracat Oranı (%)'] = (global_ihracat_hs4['Global İhracat Değeri'] / global_ihracat_hs4['Global İhracat Değeri_toplam']) * 100

# Oranları hesapla (İthalat)
global_ithalat_hs4 = global_ithalat_hs4.merge(toplam_global_textile_ithalat, on='year', how='left', suffixes=('', '_toplam'))
global_ithalat_hs4['Global İthalat Oranı (%)'] = (global_ithalat_hs4['Global İthalat Değeri'] / global_ithalat_hs4['Global İthalat Değeri_toplam']) * 100

# Yıl sütununu düzenliyoruz
global_ihracat_hs4.rename(columns={'year': 'Yıl'}, inplace=True)
global_ithalat_hs4.rename(columns={'year': 'Yıl'}, inplace=True)

# Excele çeviriyoruz
with pd.ExcelWriter("10-) Dunyada tekstilde HS4 ihracat ve ithalat kirilimlari.xlsx") as writer:
    global_ihracat_hs4[['Yıl', 'HS4', 'Global İhracat Değeri', 'Global İhracat Oranı (%)']].to_excel(writer, sheet_name='HS4 Bazında İhracat', index=False)
    global_ithalat_hs4[['Yıl', 'HS4', 'Global İthalat Değeri', 'Global İthalat Oranı (%)']].to_excel(writer, sheet_name='HS4 Bazında İthalat', index=False)

C:\Users\oguzo\AppData\Local\Temp\ipykernel_13368\1942400145.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  global_textiles['HS4'] = global_textiles['k'] // 100


In [ ]:
# 11-) Yeni tekstil urunlerinde HS2 RCA hesabi

# Türkiye kodu
turkey_code = 792

# HS2 sektör kodları dosyasını yükle
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Sektör filtresi
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2']]
hs2_list = textiles_hs2['hs2'].dropna().unique().tolist()

# Yeni ürün dosyasını yüklüyoruz
new_products_df = pd.read_excel('1-) Yeni Urunlerin Sektorleri.xlsx')
new_products_df['Sektör İsmi'] = new_products_df['Sektör İsmi'].str.strip()

# Yeni ürünleri belirliyoruz
new_products_2012 = new_products_df[new_products_df['Yıl'] == 2012]['2 Basamaklı HS Kodu'].unique().tolist()
new_products_2017 = new_products_df[new_products_df['Yıl'] == 2017]['2 Basamaklı HS Kodu'].unique().tolist()

# Tekstil sektörüne ait veriyi filtreledik (aslında gerek yoktu)
textile_trade = merged_df[(merged_df['hs2'].isin(hs2_list))]

# 2012 ve 2017 çıkışlı ürünler
def calculate_rca_for_new_products(new_products, start_year, end_year, data):
    # Yeni ürünlerin Türkiye ve dünya ticaret hacimlerini hesaplıyoruz
    new_products_trade = data[data['hs2'].isin(new_products)]
    new_products_turkiye = new_products_trade[(new_products_trade['i'] == turkey_code) & (new_products_trade['t'] >= start_year) & (new_products_trade['t'] <= end_year)]
    new_products_global = new_products_trade[(new_products_trade['t'] >= start_year) & (new_products_trade['t'] <= end_year)]

    # Türkiye ve dünya toplam ticaret hacimleri
    turkiye_total_trade = data[(data['i'] == turkey_code) & (data['t'] >= start_year) & (data['t'] <= end_year)].groupby('t')['v'].sum().reset_index()
    global_total_trade = data[(data['t'] >= start_year) & (data['t'] <= end_year)].groupby('t')['v'].sum().reset_index()

    # Yeni ürünlerin Türkiye ve dünya ticaret hacimleri
    new_products_turkiye_hacim = new_products_turkiye.groupby('t')['v'].sum().reset_index()
    new_products_global_hacim = new_products_global.groupby('t')['v'].sum().reset_index()

    # RCA 
    rca_df = new_products_turkiye_hacim.merge(turkiye_total_trade, on='t', suffixes=('_new_products', '_turkiye_total'))
    rca_df = rca_df.merge(new_products_global_hacim, on='t')
    rca_df = rca_df.merge(global_total_trade, on='t', suffixes=('_global_new_products', '_global_total'))

    rca_df['RCA'] = (rca_df['v_new_products'] / rca_df['v_turkiye_total']) / (rca_df['v_global_new_products'] / rca_df['v_global_total'])
    rca_df.rename(columns={'t': 'Yıl'}, inplace=True)
    return rca_df[['Yıl', 'v_new_products', 'v_turkiye_total', 'v_global_new_products', 'v_global_total', 'RCA']]

# RCA 
rca_2012_export = calculate_rca_for_new_products(new_products_2012, 2012, 2017, textile_trade)
rca_2017_export = calculate_rca_for_new_products(new_products_2017, 2017, 2021, textile_trade)

# İthalat için RCA
textile_trade_import = textile_trade.rename(columns={'i': 'j', 'j': 'i'})  # İthalatı analiz etmek için ters çeviriyoruz
rca_2012_import = calculate_rca_for_new_products(new_products_2012, 2012, 2017, textile_trade_import)
rca_2017_import = calculate_rca_for_new_products(new_products_2017, 2017, 2021, textile_trade_import)

with pd.ExcelWriter("11-) Yeni tekstil urunlerinde HS2 RCA hesabi.xlsx") as writer:
    rca_2012_export.to_excel(writer, sheet_name='2012 Yeni Ürünler İhracat', index=False)
    rca_2012_import.to_excel(writer, sheet_name='2012 Yeni Ürünler İthalat', index=False)
    rca_2017_export.to_excel(writer, sheet_name='2017 Yeni Ürünler İhracat', index=False)
    rca_2017_import.to_excel(writer, sheet_name='2017 Yeni Ürünler İthalat', index=False)

In [67]:
import pandas as pd

# Türkiye kodu
turkey_code = 792

# HS2 sektör kodları dosyasını yükle
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Sektör filtresi
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2', 'hs2_name']]
hs2_list = textiles_hs2['hs2'].dropna().unique().tolist()
hs2_dict = textiles_hs2.set_index('hs2')['hs2_name'].to_dict()

# Yeni ürün dosyasını yüklüyoruz
new_products_df = pd.read_excel('1-) Yeni Urunlerin Sektorleri.xlsx')
new_products_df['Sektör İsmi'] = new_products_df['Sektör İsmi'].str.strip()

# Yeni ürünleri belirliyoruz
new_products_2012 = new_products_df[new_products_df['Yıl'] == 2012]['2 Basamaklı HS Kodu'].unique().tolist()
new_products_2017 = new_products_df[new_products_df['Yıl'] == 2017]['2 Basamaklı HS Kodu'].unique().tolist()

# 2012 ve 2017 çıkışlı ürünlerin RCA hesaplaması için fonksiyon
def calculate_rca_by_hs2(new_products, start_year, end_year, data, hs2_dict):
    results = []
    for hs2_code in new_products:
        # HS2 koduna ait ticaret verileri
        hs2_trade = data[data['hs2'] == hs2_code]
        
        # Türkiye ve dünya ticaret hacimleri
        hs2_turkiye_trade = hs2_trade[(hs2_trade['i'] == turkey_code) & (hs2_trade['t'] >= start_year) & (hs2_trade['t'] <= end_year)]
        hs2_global_trade = hs2_trade[(hs2_trade['t'] >= start_year) & (hs2_trade['t'] <= end_year)]
        
        turkiye_total_trade = data[(data['i'] == turkey_code) & (data['t'] >= start_year) & (data['t'] <= end_year)].groupby('t')['v'].sum().reset_index()
        global_total_trade = data[(data['t'] >= start_year) & (data['t'] <= end_year)].groupby('t')['v'].sum().reset_index()
        
        hs2_turkiye_hacim = hs2_turkiye_trade.groupby('t')['v'].sum().reset_index()
        hs2_global_hacim = hs2_global_trade.groupby('t')['v'].sum().reset_index()
        
        # RCA hesaplama
        rca_df = hs2_turkiye_hacim.merge(turkiye_total_trade, on='t', suffixes=('_hs2', '_turkiye_total'))
        rca_df = rca_df.merge(hs2_global_hacim, on='t')
        rca_df = rca_df.merge(global_total_trade, on='t', suffixes=('_global_hs2', '_global_total'))
        
        rca_df['RCA'] = (rca_df['v_hs2'] / rca_df['v_turkiye_total']) / (rca_df['v_global_hs2'] / rca_df['v_global_total'])
        rca_df['HS2 Kodu'] = hs2_code
        rca_df['HS2 İsmi'] = hs2_dict.get(hs2_code, "Bilinmiyor")
        rca_df.rename(columns={'t': 'Yıl'}, inplace=True)
        
        results.append(rca_df[['Yıl', 'HS2 Kodu', 'HS2 İsmi', 'v_hs2', 'v_turkiye_total', 'v_global_hs2', 'v_global_total', 'RCA']])
    
    return pd.concat(results, ignore_index=True)

# RCA hesaplaması
# 2012 Yeni Ürünler
rca_2012_export = calculate_rca_by_hs2(new_products_2012, 2012, 2017, merged_df, hs2_dict)

# 2017 Yeni Ürünler
rca_2017_export = calculate_rca_by_hs2(new_products_2017, 2017, 2021, merged_df, hs2_dict)

# Excel'e kaydetme
output_file = "Yeni_Urunler_HS2_RCA_Hesabi.xlsx"
with pd.ExcelWriter(output_file) as writer:
    rca_2012_export.to_excel(writer, sheet_name='2012 Yeni Ürünler RCA', index=False)
    rca_2017_export.to_excel(writer, sheet_name='2017 Yeni Ürünler RCA', index=False)

print(f"RCA sonuçları başarıyla '{output_file}' dosyasına kaydedildi!")


RCA sonuçları başarıyla 'Yeni_Urunler_HS2_RCA_Hesabi.xlsx' dosyasına kaydedildi!
